In [ ]:
import pandas as pd
import psycopg2
import bz2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="postgres",
    password="kishiro2001"  # substitua pela sua senha real
)
cursor = conn.cursor()

In [ ]:
create_flights = """CREATE TABLE IF NOT EXISTS flights_2008 (
    Year INT,
    Month INT,
    DayofMonth INT,
    DayOfWeek INT,
    DepTime INT,
    CRSDepTime INT,
    ArrTime INT,
    CRSArrTime INT,
    UniqueCarrier TEXT,
    FlightNum INT,
    TailNum TEXT,
    ActualElapsedTime INT,
    CRSElapsedTime INT,
    AirTime INT,
    ArrDelay INT,
    DepDelay INT,
    Origin TEXT,
    Dest TEXT,
    Distance INT,
    TaxiIn INT,
    TaxiOut INT,
    Cancelled INT,
    CancellationCode TEXT,
    Diverted INT,
    CarrierDelay FLOAT,
    WeatherDelay FLOAT,
    NASDelay FLOAT,
    SecurityDelay FLOAT,
    LateAircraftDelay FLOAT
);"""
create_airports = """CREATE TABLE IF NOT EXISTS airports (
    iata TEXT PRIMARY KEY,
    airport TEXT,
    city TEXT,
    state TEXT,
    country TEXT,
    lat FLOAT,
    long FLOAT
);
"""
create_carriers = """CREATE TABLE IF NOT EXISTS carriers (
    Code TEXT PRIMARY KEY,
    Description TEXT
);
"""
create_planes = """CREATE TABLE IF NOT EXISTS plane_data (
    tailnum TEXT PRIMARY KEY,
    type TEXT,
    manufacturer TEXT,
    issue_date TEXT,
    model TEXT,
    status TEXT,
    aircraft_type TEXT,
    engine_type TEXT,
    year INT
);
"""

cursor.execute(create_flights)
cursor.execute(create_airports)
cursor.execute(create_carriers)
cursor.execute(create_planes)

conn.commit()

In [ ]:
# 1. 📥 Ler o arquivo CSV
df = pd.read_csv("dataverse_files/carriers.csv")

In [ ]:
# 2. 🧽 Limpar os dados (remover espaços desnecessários)
df["Code"] = df["Code"].str.strip()
df["Description"] = df["Description"].str.strip()

In [ ]:
# 3.    🔧 Remove linhas inválidas (sem código)
df = df.dropna(subset=["Code"])

cursor.execute("TRUNCATE TABLE carriers;")
conn.commit()

In [ ]:
# 4. 📤 Converter para CSV em memória e copiar
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False, header=False)
csv_buffer.seek(0)

cursor.copy_expert("COPY carriers FROM STDIN WITH CSV", csv_buffer)
conn.commit()

In [ ]:
# 5. ✅ Fechar conexão
cursor.close()
conn.close()
print("Importação de carriers.csv concluída com sucesso!")

Importação de carriers.csv concluída com sucesso!
